# Install Dependencies for Colab




In [13]:
!pip install -qU langgraph==0.2.44 ragas==0.2.12 nltk

In [14]:
!pip install -qU ragas==0.2.10

In [15]:
!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [16]:
!pip install python-docx

# Imports

In [17]:
import os
import getpass
from operator import itemgetter
from typing import List, Dict
import json
import requests



#LangChain, LangGraph
from langchain_openai import ChatOpenAI
from langgraph.graph import START, StateGraph, END
from typing_extensions import List, TypedDict
# from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.tools import Tool, tool
from langgraph.prebuilt import ToolNode
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams



# from langchain.tools import DuckDuckGoSearchRun



# Set Environment Variables

In [43]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


# Code Analysis Functions

In [8]:
def read_python_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        raise
    except IOError as e:
        print(f"Error reading file {file_path}: {str(e)}")
        raise
    except Exception as e:
        print(f"Unexpected error reading file {file_path}: {str(e)}")
        raise

In [ ]:
def extract_imports(code, file_path):
    try:

        # Split into lines and find imports
        import_lines = []
        for line in code.split('\n'):
            line = line.strip()
            if line.startswith('import ') or line.startswith('from '):
                import_lines.append(line)

        return import_lines

    except Exception as e:
        print(f"Error extracting imports from file {file_path}: {str(e)}")
        return []

# Tools

In [ ]:
@tool
def search_pypi(package_name: str) -> str:
    """Search PyPI for Python package information. Input should be the package name.
    Args:
        package_name: name of the package
    """
    print(f"Tool called for package: {package_name}")
    base_url = "https://pypi.org/pypi"
    try:
        try:
            response = requests.get(f"{base_url}/{package_name}/json")
            response.raise_for_status()
            info = response.json()
        except requests.RequestException as e:
            raise Exception(f"Error fetching PyPI info for {package_name}: {str(e)}")
        result =  json.dumps({
            "name": info["info"]["name"],
            "summary": info["info"]["summary"],
        })
        print(f"Tool result: {result}")
        return result
    except Exception as e:
        return f"Could not find package information: {str(e)}"

In [ ]:
print(search_pypi.args)

{'package_name': {'title': 'Package Name', 'type': 'string'}}


# Describe Whole File Agent

In [9]:
describe_code_llm = ChatOpenAI(model="gpt-4o-mini")
# system_template = """You are an expert {code_language} developer.
# Your will be given python code lines.
# Your role is to break down its components into a specific JSON format.

# Input:
# A Python file containing:
# 1- Imports
# 2- Function definitions
# 3- Execution code

# Output Format:
# The output should be a JSON with three main sections:
# 1- Imports
# {{
#   "type": "imports",
#   "description": [
#     {{"package1_name": "detailed description of package1"}},
#     {{"package2_name": "detailed description of package2"}}
#   ]
# }}

# 2- Functions
# {{
#   "functions": [
#     {{
#       "type": "function",
#       "name": "function1_name",
#       "description": "detailed explanation of function1's purpose and functionality"
#     }},
#     {{
#       "type": "function",
#       "name": "function2_name",
#       "description": "detailed explanation of function2's purpose and functionality"
#     }}
#   ]
# }}

# 3- Execution Code
# {{
#   "type": "execution",
#   "description": "comprehensive description of what the execution code does"
# }}

# Analysis Guidelines:
# 1- Imports Section:
# - Identify each imported package
# - Provide a clear, concise description of the package's purpose
# - Include the standard library or third-party nature of the package
# - Explain why the package is likely being used in this code

# 2- Functions Section:
# - List each function in the order they appear
# - Describe the function's:
# * Primary purpose
# * Input parameters
# * Return value (if any)
# * Key operations performed
# - Highlight any notable algorithms or logic within the function

# 3- Execution Code Section:
# - Describe the overall flow of the code
# - Explain how functions are called
# - Detail any data processing, computations, or side effects
# - Provide context on the script's main objective

# Important Notes:
# - Use valid JSON format for output
# - Be precise and technical in descriptions
# - Use clear, professional language
# - Avoid unnecessary verbosity
# - Focus on explaining the code's functionality and purpose
# """

system_template = """You are an expert {code_language} developer.
Your will be given python code lines.
Your role is to break down its components into a specific JSON format.

Input:
A Python file containing:
1- Imports
2- Function definitions
3- Execution code

Output Format:
The output should be a JSON with three main sections:
{{ "Imports":
    {{
      "type": "imports",
      "description": [
        {{"package1_name": "detailed description of package1"}},
        {{"package2_name": "detailed description of package2"}}
      ]
    }}


  "Functions": [
    {{
      "type": "function",
      "name": "function1_name",
      "description": "detailed explanation of function1's purpose and functionality"
    }},
    {{
      "type": "function",
      "name": "function2_name",
      "description": "detailed explanation of function2's purpose and functionality"
    }}
    ]

  'Execution':
    {{
      "type": "execution",
      "description": "comprehensive description of what the execution code does"
    }}

}}

Analysis Guidelines:
1- Imports Section:
- Identify each imported package
- Provide a clear, concise description of the package's purpose
- Include the standard library or third-party nature of the package
- Explain why the package is likely being used in this code

2- Functions Section:
- List each function in the order they appear
- Describe the function's:
* Primary purpose
* Input parameters
* Return value (if any)
* Key operations performed
- Highlight any notable algorithms or logic within the function

3- Execution Code Section:
- Describe the overall flow of the code
- Explain how functions are called
- Detail any data processing, computations, or side effects
- Provide context on the script's main objective

Important Notes:
- Use valid JSON format for output
- Be precise and technical in descriptions
- Use clear, professional language
- Avoid unnecessary verbosity
- Focus on explaining the code's functionality and purpose
"""
human_template = "{code}"

describe_code_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

describe_code_chain = (
    {"code_language": itemgetter("code_language"), "code": itemgetter("code")}
    | describe_code_prompt | describe_code_llm | StrOutputParser()
)


# chain as a node
def describe_code(state):
  # print("Starting chain function")
  last_message= state["messages"][-1]
  # print(f'last message is \n {last_message}')
  content = json.loads(last_message.content)
  # print(f'content is {content}')
  # print(type(content))
  chain_input = {"code_language": content['code_language'],
                 "code": content['code']}
  # print(f'chain_input is {chain_input}')
  # print(type(chain_input))
  response = describe_code_chain.invoke(chain_input)
  # print(f"Chain response: {response}")
  return {"messages": [AIMessage(content=response)]}




In [11]:
# file_path = "/home/melghorab/maven_nlp_course/code/Project/Code/test_code.py"
file_path = "test_code.py"
file_content = read_python_file(file_path)
file_content

'import statistics\nimport random\n\ndef generate_student_grades(num_students: int) -> list:\n    """\n    Generate a list of random student grades.\n    \n    Args:\n        num_students (int): Number of students to generate grades for\n    \n    Returns:\n        list: Random grades between 50 and 100\n    """\n    return [random.randint(50, 100) for _ in range(num_students)]\n\ndef calculate_grade_stats(grades: list) -> dict:\n    """\n    Calculate statistical metrics for a list of grades.\n    \n    Args:\n        grades (list): List of student grades\n    \n    Returns:\n        dict: Dictionary containing grade statistics\n    """\n    return {\n        \'average\': round(statistics.mean(grades), 2),\n        \'median\': statistics.median(grades),\n        \'highest\': max(grades),\n        \'lowest\': min(grades)\n    }\n\ndef main():\n    # Generate 10 student grades\n    student_grades = generate_student_grades(10)\n    \n    # Calculate and print grade statistics\n    grade_

In [ ]:
initial_state = {
    "messages": [{
        "role": "human",
        "content": json.dumps({
            "code_language": "python",
            "code": file_content
        })
    }]
}
result = compiled_code_graph.invoke(initial_state)
result

Starting chain function
last message is 
 content='{"code_language": "python", "code": "import statistics\\nimport random\\n\\ndef generate_student_grades(num_students: int) -> list:\\n    \\"\\"\\"\\n    Generate a list of random student grades.\\n    \\n    Args:\\n        num_students (int): Number of students to generate grades for\\n    \\n    Returns:\\n        list: Random grades between 50 and 100\\n    \\"\\"\\"\\n    return [random.randint(50, 100) for _ in range(num_students)]\\n\\ndef calculate_grade_stats(grades: list) -> dict:\\n    \\"\\"\\"\\n    Calculate statistical metrics for a list of grades.\\n    \\n    Args:\\n        grades (list): List of student grades\\n    \\n    Returns:\\n        dict: Dictionary containing grade statistics\\n    \\"\\"\\"\\n    return {\\n        \'average\': round(statistics.mean(grades), 2),\\n        \'median\': statistics.median(grades),\\n        \'highest\': max(grades),\\n        \'lowest\': min(grades)\\n    }\\n\\ndef main():\\n

{'messages': [HumanMessage(content='{"code_language": "python", "code": "import statistics\\nimport random\\n\\ndef generate_student_grades(num_students: int) -> list:\\n    \\"\\"\\"\\n    Generate a list of random student grades.\\n    \\n    Args:\\n        num_students (int): Number of students to generate grades for\\n    \\n    Returns:\\n        list: Random grades between 50 and 100\\n    \\"\\"\\"\\n    return [random.randint(50, 100) for _ in range(num_students)]\\n\\ndef calculate_grade_stats(grades: list) -> dict:\\n    \\"\\"\\"\\n    Calculate statistical metrics for a list of grades.\\n    \\n    Args:\\n        grades (list): List of student grades\\n    \\n    Returns:\\n        dict: Dictionary containing grade statistics\\n    \\"\\"\\"\\n    return {\\n        \'average\': round(statistics.mean(grades), 2),\\n        \'median\': statistics.median(grades),\\n        \'highest\': max(grades),\\n        \'lowest\': min(grades)\\n    }\\n\\ndef main():\\n    # Generate 

In [ ]:
result['messages'][-1].content

'```json\n{\n  "imports": {\n    "type": "imports",\n    "description": [\n      {\n        "statistics": "This is a standard library in Python that provides functions to calculate mathematical statistics of numeric data. It is being used here to compute statistical metrics such as mean and median for the generated student grades."\n      },\n      {\n        "random": "This is a standard library in Python that implements pseudo-random number generators for various distributions. It is used in this code to generate random grades for the students in the specified range."\n      }\n    ]\n  },\n  "functions": [\n    {\n      "type": "function",\n      "name": "generate_student_grades",\n      "description": "This function generates a list of random student grades. It takes a single parameter, num_students (int), which specifies how many student grades to generate. It returns a list of random integers representing grades within the range of 50 to 100. The function uses list comprehension 

# Write Whole Document

## Process documenter result

In [ ]:
json_content = result['messages'][-1].content
json_content = json_content[json_content.find("{"):json_content.rfind("}")+1].strip()
json_content = json.loads(json_content)
json_content


{'imports': {'type': 'imports',
  'description': [{'statistics': 'This is a standard library in Python that provides functions to calculate mathematical statistics of numeric data. It is being used here to compute statistical metrics such as mean and median for the generated student grades.'},
   {'random': 'This is a standard library in Python that implements pseudo-random number generators for various distributions. It is used in this code to generate random grades for the students in the specified range.'}]},
 'functions': [{'type': 'function',
   'name': 'generate_student_grades',
   'description': 'This function generates a list of random student grades. It takes a single parameter, num_students (int), which specifies how many student grades to generate. It returns a list of random integers representing grades within the range of 50 to 100. The function uses list comprehension to create the list of grades by calling random.randint for the specified number of students.'},
  {'type'

## Document Writer Agent

In [12]:
documenter_llm = ChatOpenAI(model="gpt-4o-mini")
system_template = """
Create a comprehensive Word document from the provided JSON input describing a Python script.

Document Requirements:
1. Title should reflect the script's primary purpose
2. Organize content into logical sections:
   - Imports
   - Functions
   - Execution Mechanism
   - Optional: Technical Insights and Potential Improvements

For Each Section:
- Explain the purpose and functionality
- Provide technical details
- Use professional technical writing style
- Include function signatures and parameter descriptions
- Break down complex descriptions into clear, concise points

Formatting Guidelines:
- Use a clean, professional Word document template
- Ensure consistent font and spacing
- Use bold text for emphasis
- Create bulleted or numbered lists for detailed explanations
- Include any available descriptions or comments from the JSON input

Specific Section Handling:
- Imports: Explain each imported library's purpose and specific use in the script
- Functions:
  - Provide detailed function signatures
  - Explain input parameters
  - Describe return values
  - Break down the function's purpose and mechanism
- Execution: Explain how the script is intended to run and its primary workflow

Additional Recommendations:
- If the JSON includes type information, incorporate it into the documentation
- Add context to explain the script's overall purpose
- Suggest potential improvements or extensions if the JSON provides enough context

Final Output:
- Fully formatted .docx file
- Comprehensive explanation of the script
- Technical yet readable documentation
- Output should be the document content only without any introduction

"""

human_template = "{content}"

documenter_llm_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

documenter_chain = (
    {"content": itemgetter("content")}
    | documenter_llm_prompt
    | documenter_llm
    | StrOutputParser()
)

def write_document_content(state):
  print(state)
  json_content = state['messages'][-1].content
  json_content = json_content[json_content.find("{"):json_content.rfind("}")+1].strip()
  json_content = json.loads(json_content)
  document_response = documenter_chain.invoke({"content": json_content})
  return {"messages": [AIMessage(content=document_response)]}


In [ ]:
document_response = documenter_chain.invoke({"content": json_content})
print(document_response)

# Student Grades Statistics Generator

## Imports
The script begins by importing necessary libraries that facilitate various functionalities:

- **statistics**: 
  - This is a standard library in Python that provides functions to compute mathematical statistics of numeric data. 
  - **Usage**: Employed within the script to compute key statistical metrics such as the mean and median for the generated student grades.

- **random**: 
  - This is another standard library in Python that implements pseudo-random number generators for various distributions. 
  - **Usage**: Utilized in the script to generate random grades within a specified range of values.

## Functions

### `generate_student_grades(num_students: int) -> list`
- **Description**: 
  - Generates a list of random student grades for a specified number of students.
- **Parameters**: 
  - `num_students`: An integer that indicates how many student grades should be generated.
- **Returns**: 
  - A list of random integers, representin

## Write content to file

In [13]:
from docx import Document
import re

def write_to_docx(state):
    text = state['messages'][-1].content
    filename = 'generated_documentation.docx'
    doc = Document()

    lines = text.split("\n")
    for line in lines:
        if line.startswith("### "):
            doc.add_heading(line[4:], level=3)
        elif line.startswith("## "):
            doc.add_heading(line[3:], level=2)
        elif line.startswith("# "):
            doc.add_heading(line[2:], level=1)
        elif "**" in line:
            bold_parts = re.split(r"(\*\*.*?\*\*)", line)
            para = doc.add_paragraph()
            for part in bold_parts:
                if part.startswith("**") and part.endswith("**"):
                    para.add_run(part[2:-2]).bold = True
                else:
                    para.add_run(part)
        else:
            doc.add_paragraph(line)

    # Save document
    doc.save(filename)
    return {"messages": [SystemMessage(content=['Saved document successfully'])]}


In [ ]:
file = write_to_docx(document_response)

# Construct Graph

In [14]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]
  # messages: Annotated[List[BaseMessage], operator.add]

In [15]:
uncompiled_code_graph = StateGraph(AgentState)
uncompiled_code_graph.add_node("code_agent", describe_code)
uncompiled_code_graph.add_node("write_content_agent", write_document_content)
uncompiled_code_graph.add_node("write_document", write_to_docx)

uncompiled_code_graph.set_entry_point("code_agent")
uncompiled_code_graph.add_edge("code_agent", "write_content_agent")
uncompiled_code_graph.add_edge("write_content_agent", "write_document")

compiled_code_graph = uncompiled_code_graph.compile()

In [17]:
initial_state = {
    "messages": [{
        "role": "human",
        "content": json.dumps({
            "code_language": "python",
            "code": file_content
        })
    }]
}
# result = compiled_code_graph.invoke(initial_state)
# result

In [24]:
async for chunk in compiled_code_graph.astream(initial_state, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'code_agent'
[AIMessage(content='```json\n{\n  "imports": {\n    "type": "imports",\n    "description": [\n      {\n        "statistics": "A standard library module for calculating statistics such as mean, median, mode, and variance. It is used here to perform statistical calculations on student grades."\n      },\n      {\n        "random": "A standard library module that implements pseudo-random number generators and is used to generate random grades between specified limits."\n      }\n    ]\n  },\n  "functions": [\n    {\n      "type": "function",\n      "name": "generate_student_grades",\n      "description": "This function generates a list of random grades for a specified number of students. It takes an integer \'num_students\' as input and returns a list of grades, each randomly chosen between 50 and 100."\n    },\n    {\n      "type": "function",\n      "name": "calculate_grade_stats",\n      "description": "This function calculates statistical metri

In [18]:
result = compiled_code_graph.invoke(initial_state)
result

{'messages': [HumanMessage(content='{"code_language": "python", "code": "import statistics\\nimport random\\n\\ndef generate_student_grades(num_students: int) -> list:\\n    \\"\\"\\"\\n    Generate a list of random student grades.\\n    \\n    Args:\\n        num_students (int): Number of students to generate grades for\\n    \\n    Returns:\\n        list: Random grades between 50 and 100\\n    \\"\\"\\"\\n    return [random.randint(50, 100) for _ in range(num_students)]\\n\\ndef calculate_grade_stats(grades: list) -> dict:\\n    \\"\\"\\"\\n    Calculate statistical metrics for a list of grades.\\n    \\n    Args:\\n        grades (list): List of student grades\\n    \\n    Returns:\\n        dict: Dictionary containing grade statistics\\n    \\"\\"\\"\\n    return {\\n        \'average\': round(statistics.mean(grades), 2),\\n        \'median\': statistics.median(grades),\\n        \'highest\': max(grades),\\n        \'lowest\': min(grades)\\n    }\\n\\ndef main():\\n    # Generate 

{'messages': [HumanMessage(content='{"code_language": "python", "code": "import statistics\\nimport random\\n\\ndef generate_student_grades(num_students: int) -> list:\\n    \\"\\"\\"\\n    Generate a list of random student grades.\\n    \\n    Args:\\n        num_students (int): Number of students to generate grades for\\n    \\n    Returns:\\n        list: Random grades between 50 and 100\\n    \\"\\"\\"\\n    return [random.randint(50, 100) for _ in range(num_students)]\\n\\ndef calculate_grade_stats(grades: list) -> dict:\\n    \\"\\"\\"\\n    Calculate statistical metrics for a list of grades.\\n    \\n    Args:\\n        grades (list): List of student grades\\n    \\n    Returns:\\n        dict: Dictionary containing grade statistics\\n    \\"\\"\\"\\n    return {\\n        \'average\': round(statistics.mean(grades), 2),\\n        \'median\': statistics.median(grades),\\n        \'highest\': max(grades),\\n        \'lowest\': min(grades)\\n    }\\n\\ndef main():\\n    # Generate 

In [ ]:
file_path = result['messages'][1].content
file_path

'```json\n{\n  "imports": {\n    "type": "imports",\n    "description": [\n      {\n        "statistics": "A standard library in Python used for statistical calculations and metrics. This module is likely being used to compute various statistical properties of student grades."\n      },\n      {\n        "random": "A standard library in Python that generates pseudo-random numbers. This module is used to create random student grades within a specified range."\n      }\n    ]\n  },\n  "functions": [\n    {\n      "type": "function",\n      "name": "generate_student_grades",\n      "description": "Generates a list of random student grades. It takes an integer parameter representing the number of students and returns a list of integers, each representing a grade between 50 and 100."\n    },\n    {\n      "type": "function",\n      "name": "calculate_grade_stats",\n      "description": "Calculates statistical metrics for a list of grades. It takes a list of grades as input and returns a dic

# Describe Imports Agent

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_tool
# search_packages_tools = [convert_to_openai_tool(search_pypi)]

search_packages_tools = [search_pypi]
# tools=[
#         {
#             "type": "function",
#             "function": {
#                 "name": "search_pypi",
#                 "description": "Search PyPI for Python package information. Input should be the package name.",
#                 "parameters": {
#                     "type": "object",
#                     "properties": {
#                         "package_name": {"type": "string", "description": "Package Name"},

#                     },
#                     "required": ["package_name"],
#                 },
#             },
#         }
#     ]

In [ ]:
describe_imports_llm = ChatOpenAI(model="gpt-4o-mini")
# describe_imports_llm = describe_imports_llm.bind_tools(tools = search_packages_tools, tool_choice="required")

In [ ]:
system_template = """You are an expert {code_language} developer.
Your will be given code lines that import packages.
Your role is to give a brief description of each package

You have access to the following tool and you MUST use it:
search_pypi: Use this to get information about Python packages from PyPI.

For each import:
1. Extract the main package name
2. Use the search_pypi tool to get package information by calling "search_pypi(package_name)"
3. Combine the information into a clear description
4. If the retuned value of tool is empty use your own knowledge
5. If you have no knowledge for this package then it's description should be "I don't know details about this package"

You must respond in the following JSON format:
{{"Imported_Packages": [
    {{"name": "package1", "desc": "brief description of package1"}},
    {{"name": "package2", "desc": "brief description of package2"}}
]}}

Rules for the output:
1. Use valid JSON format
2. Package names should be the exact names from the imports
3. Descriptions should be brief and clear
4. Do not include any text outside the JSON structure
"""

# system_template = """You are an expert {code_language} developer.
# Your will be given code lines that import packages.
# Your role is to give a brief description of each package

# You can use the search_pypi tool, but it's optional.
# If you know about a package, you can describe it directly.
# If you're unsure, you can use search_pypi to get more information.

# You must respond in the following JSON format:
# {{"Imported_Packages": [
#     {{"name": "package1", "desc": "brief description of package1"}},
#     {{"name": "package2", "desc": "brief description of package2"}}
# ]}}

# Rules for the output:
# 1. Use valid JSON format
# 2. Package names should be the exact names from the imports
# 3. Descriptions should be brief and clear
# 4. Do not include any text outside the JSON structure
# """
human_template = "{imports}"

describe_imports_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

In [ ]:

describe_imports_chain = (
    {"code_language": itemgetter("code_language"), "imports": itemgetter("imports")}
    | describe_imports_prompt | describe_imports_llm | StrOutputParser()
)

In [ ]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

In [ ]:
# chain as a node
def call_imports_chain(state):
  print("Starting chain function")
  last_message= state["messages"][-1]
  print(f'last message is \n {last_message}')
  content = json.loads(last_message.content)
  print(f'content is {content}')
  print(type(content))
  chain_input = {"code_language": content['code_language'],
                 "imports": content['imports']}
  print(f'chain_input is {chain_input}')
  print(type(chain_input))
  response = describe_imports_chain.invoke(chain_input)
  print(f"Chain response: {response}")
  return {"messages": [AIMessage(content=response)]}
#   return {
#           "messages": [{
#               "role": "assistant",
#               "content": response
#           }]
#       }

In [ ]:
# bind model to tool or ToolNode
imports_tool_node = ToolNode(search_packages_tools)


In [ ]:
# construct graph and compile
uncompiled_imports_graph = StateGraph(AgentState)
uncompiled_imports_graph.add_node("imports_agent", call_imports_chain)
uncompiled_imports_graph.add_node("imports_action", imports_tool_node)
uncompiled_imports_graph.set_entry_point("imports_agent")
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "imports_action"

  return END

uncompiled_imports_graph.add_conditional_edges(
    "imports_agent",
    should_continue
)

uncompiled_imports_graph.add_edge("imports_action", "imports_agent")

compiled_imports_graph = uncompiled_imports_graph.compile()


In [ ]:
# invoke graph

In [ ]:
file_path = "/home/melghorab/maven_nlp_course/code/Project/Code/test_code.py"
file_content = read_python_file(file_path)
file_content

'import os\nimport getpass\nfrom operator import itemgetter\n\n\n#LangChain, LangGraph\nfrom langchain_openai import ChatOpenAI\nfrom langgraph.graph import START, StateGraph\nfrom typing_extensions import List, TypedDict\nfrom langchain_core.documents import Document\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain.schema.output_parser import StrOutputParser\n\ndef read_python_file(file_path):\n    try:\n        with open(file_path, \'r\', encoding=\'utf-8\') as f:\n            content = f.read()\n        return content\n    except FileNotFoundError:\n        print(f"File not found: {file_path}")\n        raise\n    except IOError as e:\n        print(f"Error reading file {file_path}: {str(e)}")\n        raise\n    except Exception as e:\n        print(f"Unexpected error reading file {file_path}: {str(e)}")\n        raise'

In [ ]:
imports = extract_imports(file_content, file_path)
imports

['import os',
 'import getpass',
 'from operator import itemgetter',
 'from langchain_openai import ChatOpenAI',
 'from langgraph.graph import START, StateGraph',
 'from typing_extensions import List, TypedDict',
 'from langchain_core.documents import Document',
 'from langchain_core.prompts import ChatPromptTemplate',
 'from langchain.schema.output_parser import StrOutputParser']

In [ ]:
# input_message = HumanMessage(content=json.dumps({
#     "code_language": "python",
#     "imports": imports
# }))

# result = compiled_imports_graph.invoke({
#             "messages": [input_message]
#         })

# result

# initial_state = {
#     "messages": [{
#         "code_language": "python",
#         "imports": imports
#     }]
# }

initial_state = {
    "messages": [{
        "role": "human",
        "content": json.dumps({
            "code_language": "python",
            "imports": imports
        })
    }]
}
result = compiled_imports_graph.invoke(initial_state)
result

# inputs = {"messages" : [HumanMessage(content={"code_language": "python", "imports": imports})]}

Starting chain function
last message is 
 content='{"code_language": "python", "imports": ["import os", "import getpass", "from operator import itemgetter", "from langchain_openai import ChatOpenAI", "from langgraph.graph import START, StateGraph", "from typing_extensions import List, TypedDict", "from langchain_core.documents import Document", "from langchain_core.prompts import ChatPromptTemplate", "from langchain.schema.output_parser import StrOutputParser"]}' additional_kwargs={} response_metadata={} id='86b01ffc-c0cf-4f76-87d6-81aed0ddb10e'
content is {'code_language': 'python', 'imports': ['import os', 'import getpass', 'from operator import itemgetter', 'from langchain_openai import ChatOpenAI', 'from langgraph.graph import START, StateGraph', 'from typing_extensions import List, TypedDict', 'from langchain_core.documents import Document', 'from langchain_core.prompts import ChatPromptTemplate', 'from langchain.schema.output_parser import StrOutputParser']}
<class 'dict'>
chain_

{'messages': [HumanMessage(content='{"code_language": "python", "imports": ["import os", "import getpass", "from operator import itemgetter", "from langchain_openai import ChatOpenAI", "from langgraph.graph import START, StateGraph", "from typing_extensions import List, TypedDict", "from langchain_core.documents import Document", "from langchain_core.prompts import ChatPromptTemplate", "from langchain.schema.output_parser import StrOutputParser"]}', additional_kwargs={}, response_metadata={}, id='86b01ffc-c0cf-4f76-87d6-81aed0ddb10e'),
  AIMessage(content='{"Imported_Packages": [\n    {"name": "os", "desc": "A standard library that provides a way of using operating system-dependent functionality like reading or writing to the file system."},\n    {"name": "getpass", "desc": "A standard library that provides a way to securely handle password prompts without echoing input."},\n    {"name": "operator", "desc": "A standard library that exports a set of functions corresponding to the intrin

In [ ]:
async for chunk in compiled_imports_graph.astream(initial_state, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Starting chain function
last message is 
 content='{"code_language": "python", "imports": ["import os", "import getpass", "from operator import itemgetter", "from langchain_openai import ChatOpenAI", "from langgraph.graph import START, StateGraph", "from typing_extensions import List, TypedDict", "from langchain_core.documents import Document", "from langchain_core.prompts import ChatPromptTemplate", "from langchain.schema.output_parser import StrOutputParser"]}' additional_kwargs={} response_metadata={} id='feeaeda4-a5a5-454f-bd0d-061b7bb9793b'
content is {'code_language': 'python', 'imports': ['import os', 'import getpass', 'from operator import itemgetter', 'from langchain_openai import ChatOpenAI', 'from langgraph.graph import START, StateGraph', 'from typing_extensions import List, TypedDict', 'from langchain_core.documents import Document', 'from langchain_core.prompts import ChatPromptTemplate', 'from langchain.schema.output_parser import StrOutputParser']}
<class 'dict'>
chain_

In [ ]:
# response=describe_imports_chain.invoke({"code_language": "python", "imports": imports})
response=describe_imports_chain.invoke({'code_language': 'python', 'imports': ['import os', 'import getpass', 'from operator import itemgetter', 'from langchain_openai import ChatOpenAI', 'from langgraph.graph import START, StateGraph']})
response

''

In [ ]:
response = describe_imports_llm.invoke("What tools do you have available?")
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gh46jGv1jZD196HTw3WxnaNZ', 'function': {'arguments': '{"package_name":"requests"}', 'name': 'search_pypi'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 72, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d8ebc589-e82e-42d4-9c18-c020c81043fa-0', tool_calls=[{'name': 'search_pypi', 'args': {'package_name': 'requests'}, 'id': 'call_gh46jGv1jZD196HTw3WxnaNZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 72, 'output_tokens': 18, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio

In [ ]:
print("Testing tool directly:")
result = search_pypi("pandas")
print(f"Tool result: {result}\n")

Testing tool directly:
Tool called for package: pandas
Tool result: {"name": "pandas", "summary": "Powerful data structures for data analysis, time series, and statistics"}
Tool result: {"name": "pandas", "summary": "Powerful data structures for data analysis, time series, and statistics"}



In [ ]:
print(describe_imports_llm)


bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7f77e94f6ce0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f77e94f7ac0>, root_client=<openai.OpenAI object at 0x7f77e94ec6a0>, root_async_client=<openai.AsyncOpenAI object at 0x7f77e94f4df0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')) kwargs={'tools': [{'type': 'function', 'function': {'name': 'search_pypi', 'description': 'Search PyPI for Python package information. Input should be the package name.\n    Args: \n        package_name: name of the package', 'parameters': {'properties': {'package_name': {'type': 'string'}}, 'required': ['package_name'], 'type': 'object'}}}], 'tool_choice': 'required'} config={} config_factories=[]


In [ ]:
result

{'messages': [HumanMessage(content='{"code_language": "python", "imports": ["import os", "import getpass", "from operator import itemgetter", "from langchain_openai import ChatOpenAI", "from langgraph.graph import START, StateGraph", "from typing_extensions import List, TypedDict", "from langchain_core.documents import Document", "from langchain_core.prompts import ChatPromptTemplate", "from langchain.schema.output_parser import StrOutputParser"]}', additional_kwargs={}, response_metadata={}, id='86b01ffc-c0cf-4f76-87d6-81aed0ddb10e'),
  AIMessage(content='{"Imported_Packages": [\n    {"name": "os", "desc": "A standard library that provides a way of using operating system-dependent functionality like reading or writing to the file system."},\n    {"name": "getpass", "desc": "A standard library that provides a way to securely handle password prompts without echoing input."},\n    {"name": "operator", "desc": "A standard library that exports a set of functions corresponding to the intrin

# Save description blocks as chunks in database

In [28]:
text = result['messages'][1].content
text


'```json\n{\n  "Imports": {\n    "type": "imports",\n    "description": [\n      {\n        "statistics": "A standard library module in Python for calculating statistical metrics such as mean, median, and variance. This module is used to analyze the list of student grades."\n      },\n      {\n        "random": "A standard library module in Python for generating random numbers. This module is used to create random student grades within a specified range."\n      }\n    ]\n  },\n  "Functions": {\n    "functions": [\n      {\n        "type": "function",\n        "name": "generate_student_grades",\n        "description": "This function generates a list of random student grades. It takes a single parameter, num_students, which specifies how many student grades to generate, and returns a list of random integers representing grades between 50 and 100."\n      },\n      {\n        "type": "function",\n        "name": "calculate_grade_stats",\n        "description": "This function calculates s

In [19]:
qdrant_client = QdrantClient(":memory:")  # Use in-memory or provide host/port

# Define embedding model
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")


<ipython-input-19-1cf1193a7d88>:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")


In [20]:
chunks = result['messages'][1].content
chunks = chunks[chunks.find("{"):chunks.rfind("}")+1].strip()
chunks = json.loads(chunks)
chunks

{'Imports': {'type': 'imports',
  'description': [{'statistics': 'A standard library in Python that provides functions for calculating mathematical statistics of numeric data. This package is used for computing metrics like mean, median, and other statistical measures on the student grades.'},
   {'random': 'A standard library in Python that implements pseudo-random number generators for various distributions. This package is used to generate random integers for student grades within a specified range.'}]},
 'Functions': [{'type': 'function',
   'name': 'generate_student_grades',
   'description': 'Generates a list of random student grades based on the number of students specified. It takes an integer as input representing the number of students and returns a list of integers representing grades, which are randomly generated between 50 and 100.'},
  {'type': 'function',
   'name': 'calculate_grade_stats',
   'description': 'Calculates and returns statistical metrics for a given list of

In [21]:
chunks_list = []
for key in chunks:
    if isinstance(chunks[key], dict):
        chunks_list.append(chunks[key])
    elif isinstance(chunks[key], list):
        for value in chunks[key]:
            chunks_list.append(value)
chunks_list

[{'type': 'imports',
  'description': [{'statistics': 'A standard library in Python that provides functions for calculating mathematical statistics of numeric data. This package is used for computing metrics like mean, median, and other statistical measures on the student grades.'},
   {'random': 'A standard library in Python that implements pseudo-random number generators for various distributions. This package is used to generate random integers for student grades within a specified range.'}]},
 {'type': 'function',
  'name': 'generate_student_grades',
  'description': 'Generates a list of random student grades based on the number of students specified. It takes an integer as input representing the number of students and returns a list of integers representing grades, which are randomly generated between 50 and 100.'},
 {'type': 'function',
  'name': 'calculate_grade_stats',
  'description': 'Calculates and returns statistical metrics for a given list of grades. It takes a list of gr

In [39]:
len(chunks_list)

5

In [22]:
from langchain.schema import Document
docs = [
    Document(
        page_content=f"{chunk.get('type', '')} - {chunk.get('name', '')} - {chunk.get('description', '')}",  # Content for the model
        metadata={**chunk}  # Store metadata, but don't put embeddings here
    )
    for chunk in chunks_list
]

docs

[Document(metadata={'type': 'imports', 'description': [{'statistics': 'A standard library in Python that provides functions for calculating mathematical statistics of numeric data. This package is used for computing metrics like mean, median, and other statistical measures on the student grades.'}, {'random': 'A standard library in Python that implements pseudo-random number generators for various distributions. This package is used to generate random integers for student grades within a specified range.'}]}, page_content="imports -  - [{'statistics': 'A standard library in Python that provides functions for calculating mathematical statistics of numeric data. This package is used for computing metrics like mean, median, and other statistical measures on the student grades.'}, {'random': 'A standard library in Python that implements pseudo-random number generators for various distributions. This package is used to generate random integers for student grades within a specified range.'}]

In [65]:
import pickle
with open("description_chunks.pkl", "wb") as pickle_file:
    pickle.dump(docs, pickle_file)

In [23]:
embedding_dim = 1536
qdrant_client.create_collection(
    collection_name="description_rag_data",
    vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
)

# Store in Qdrant with explicit embeddings
vectorstore = Qdrant(qdrant_client, collection_name="description_rag_data", embeddings=embedding_model)
vectorstore.add_documents(docs)


# Define the retriever
qdrant_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# # Define the prompt template
# rag_prompt = PromptTemplate.from_template(
#     "Given the following retrieved documents:\n{context}\nAnswer the question: {question}"
# )

# # Define the LLM model (OpenAI ChatGPT)
# openai_chat_model = ChatOpenAI(model_name="gpt-4", temperature=0.7)

# # Define the RAG chain
# rag_chain = (
#     {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
#     | rag_prompt
#     | openai_chat_model
#     | StrOutputParser()
# )

# # Example query
# query = {"question": "What are the company policies?"}
# response = rag_chain.invoke(query)

# # Print response
# print("\nAI Response:", response)

<ipython-input-23-58055bb816db>:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(qdrant_client, collection_name="description_rag_data", embeddings=embedding_model)


In [ ]:
# from langchain_openai.embeddings import OpenAIEmbeddings

# embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# embedding_dim = 1536

# from langchain_qdrant import QdrantVectorStore
# from qdrant_client import QdrantClient
# from qdrant_client.http.models import Distance, VectorParams

# client = QdrantClient(":memory:")

# client.create_collection(
#     collection_name="lcel_doc_v1",
#     vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
# )

# vector_store = QdrantVectorStore(
#     client=client,
#     collection_name="lcel_doc_v1",
#     embedding=embedding_model,
# )

# _ = vector_store.add_texts(texts=chunks)

# retriever = vector_store.as_retriever(search_kwargs={"k": 2})



# Call main LLM

In [ ]:
imports_result = result['messages']

In [ ]:
result['messages'][-1].content

str

In [24]:
main_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
system_template = """You are an expert {code_language} developer.
Your role is to answer user's questions about code and its description that will be given to you in context

Rules for the output:
1. Don't answer out of context questions
2. Provide a single, clear response using only the given context
3. If needed, structure long responses in lists and sections

<context>
{context}
</context>
"""

human_template = "{query}"
main_llm_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

# main_chain = (
#     {"code_language": itemgetter("code_language"),"context": itemgetter("context"), "query": itemgetter("query")}
#     | main_llm_prompt | main_llm | StrOutputParser()
# )

main_chain = (
    {"context": itemgetter("query") | qdrant_retriever, "code_language": itemgetter("code_language"), "query": itemgetter("query"), }
    | main_llm_prompt
    | main_llm
    | StrOutputParser()
)



In [ ]:
context = result['messages'][-1].content
context

'{"Imported_Packages": [\n    {"name": "os", "desc": "The os module provides a way of using operating system dependent functionality like reading or writing to the file system."},\n    {"name": "getpass", "desc": "The getpass module provides a secure way to handle password prompts and enables users to input passwords without echoing them to the console."},\n    {"name": "operator", "desc": "The operator module provides a set of efficient functions corresponding to the intrinsic operators of Python, allowing for functional programming."},\n    {"name": "langchain_openai", "desc": "LangChain for OpenAI integrates OpenAI models into workflows, enabling building applications that leverage language models."},\n    {"name": "langgraph", "desc": "No knowledge of this package"},\n    {"name": "typing_extensions", "desc": "The typing_extensions module provides backports of type hints from the standard library\'s typing module, making them available in earlier Python versions."},\n    {"name": "

In [25]:
# response=main_chain.invoke({'code_language': 'Python', 'context': context, 'query': 'What are the packages used in this code?'})
# response
# response=main_chain.invoke({'code_language': 'python', 'context': context, 'query': 'What are the packages used in this code?'})
# response

# inputs = {
#     'code_language': 'Python',
#     'context': context,
#     'query': 'What are the packages used in this code?'
# }

inputs = {
    'code_language': 'Python',
    'query': 'what is the use of function "generate_student_grades"?'
}
# print(inputs)  # Debugging step
response = main_chain.invoke(inputs)
print(response)
# response


The function "generate_student_grades" is used to generate a list of random student grades. It takes an integer input representing the number of students and returns a list of integers that represent grades, which are randomly generated between 50 and 100.


# Write in Document

In [ ]:
docs

[Document(metadata={'type': 'Imported Packages', 'name': 'Imported Packages', 'content': '{"Imported_Packages": [\n    {"name": "os", "desc": "The os module provides a way of using operating system dependent functionality like reading or writing to the file system."},\n    {"name": "getpass", "desc": "The getpass module provides a secure way to handle password prompts and enables users to input passwords without echoing them to the console."},\n    {"name": "operator", "desc": "The operator module provides a set of efficient functions corresponding to the intrinsic operators of Python, allowing for functional programming."},\n    {"name": "langchain_openai", "desc": "LangChain for OpenAI integrates OpenAI models into workflows, enabling building applications that leverage language models."},\n    {"name": "langgraph", "desc": "No knowledge of this package"},\n    {"name": "typing_extensions", "desc": "The typing_extensions module provides backports of type hints from the standard libra

In [ ]:
#extract description chunks from database
collection_name = "description_rag_data"
all_points = qdrant_client.scroll(collection_name=collection_name, limit=1000)[0]  # Adjust limit if needed
one_chunk = all_points[0].payload
one_chunk

{'page_content': 'Imported Packages - Imported Packages - {"Imported_Packages": [\n    {"name": "os", "desc": "The os module provides a way of using operating system dependent functionality like reading or writing to the file system."},\n    {"name": "getpass", "desc": "The getpass module provides a secure way to handle password prompts and enables users to input passwords without echoing them to the console."},\n    {"name": "operator", "desc": "The operator module provides a set of efficient functions corresponding to the intrinsic operators of Python, allowing for functional programming."},\n    {"name": "langchain_openai", "desc": "LangChain for OpenAI integrates OpenAI models into workflows, enabling building applications that leverage language models."},\n    {"name": "langgraph", "desc": "No knowledge of this package"},\n    {"name": "typing_extensions", "desc": "The typing_extensions module provides backports of type hints from the standard library\'s typing module, making them

In [ ]:
@tool
def write_document(
    content: Annotated[str, "Text content to be written into the document."],
    file_name: Annotated[str, "File path to save the document."],
) -> Annotated[str, "Path of the saved document file."]:
    """Create and save a text document."""
    with (WORKING_DIRECTORY / file_name).open("w") as file:
        file.write(content)
    return f"Document saved to {file_name}"



In [ ]:
one_chunk

{'page_content': 'Imported Packages - Imported Packages - {"Imported_Packages": [\n    {"name": "os", "desc": "The os module provides a way of using operating system dependent functionality like reading or writing to the file system."},\n    {"name": "getpass", "desc": "The getpass module provides a secure way to handle password prompts and enables users to input passwords without echoing them to the console."},\n    {"name": "operator", "desc": "The operator module provides a set of efficient functions corresponding to the intrinsic operators of Python, allowing for functional programming."},\n    {"name": "langchain_openai", "desc": "LangChain for OpenAI integrates OpenAI models into workflows, enabling building applications that leverage language models."},\n    {"name": "langgraph", "desc": "No knowledge of this package"},\n    {"name": "typing_extensions", "desc": "The typing_extensions module provides backports of type hints from the standard library\'s typing module, making them

In [ ]:
input_text = f"type: {one_chunk['metadata']['type']} \nname: {one_chunk['metadata']['name']} \ncontent: {one_chunk['metadata']['content']}"
print(input_text)

type: Imported Packages 
name: Imported Packages 
content: {"Imported_Packages": [
    {"name": "os", "desc": "The os module provides a way of using operating system dependent functionality like reading or writing to the file system."},
    {"name": "getpass", "desc": "The getpass module provides a secure way to handle password prompts and enables users to input passwords without echoing them to the console."},
    {"name": "operator", "desc": "The operator module provides a set of efficient functions corresponding to the intrinsic operators of Python, allowing for functional programming."},
    {"name": "langchain_openai", "desc": "LangChain for OpenAI integrates OpenAI models into workflows, enabling building applications that leverage language models."},
    {"name": "langgraph", "desc": "No knowledge of this package"},
    {"name": "typing_extensions", "desc": "The typing_extensions module provides backports of type hints from the standard library's typing module, making them avail

In [ ]:
documenter_llm = ChatOpenAI(model="gpt-4o-mini")
system_template = """You are an expert code documenter.
Your role is to write a well structured document that describes code functionality.

From the given context:
1- type: is the type of the code block (funciton, class, ..)
2- name: is the name of the code block
3- content: is the description of the code block

Instructions:
Write a docx document with the following structure Heading 1(type) -> Heading 2(name) -> content

Rules for the output:
1. Don't write information out of context
2. If needed, structure long responses in lists and sections

<context>
{context}
</context>
"""

human_template = "{code_block}"

documenter_llm_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
])

documenter_chain = (
    {"context": itemgetter("context")}
    | documenter_llm_prompt
    | documenter_llm
    | StrOutputParser()
)


In [ ]:
document_response = documenter_chain.invoke({"context": input_text})
print(document_response)


# Imported Packages 

## Imported Packages 

The following is a list of imported packages used in the code, detailing their names and descriptions:

1. **os**
   - The os module provides a way of using operating system dependent functionality like reading or writing to the file system.

2. **getpass**
   - The getpass module provides a secure way to handle password prompts and enables users to input passwords without echoing them to the console.

3. **operator**
   - The operator module provides a set of efficient functions corresponding to the intrinsic operators of Python, allowing for functional programming.

4. **langchain_openai**
   - LangChain for OpenAI integrates OpenAI models into workflows, enabling building applications that leverage language models.

5. **langgraph**
   - No knowledge of this package.

6. **typing_extensions**
   - The typing_extensions module provides backports of type hints from the standard library's typing module, making them available in earlier Pytho

In [ ]:
from docx import Document
import re

# @tool
def write_to_docx(documentation_text: str) -> str:
    """
    Writes the AI-generated documentation to a .docx file and returns the file path.
    """
    doc = Document()
    # doc.add_heading("Code Documentation", level=1)

    lines = documentation_text.split("\n")
    for line in lines:
        if line.startswith("# "):  # Section Heading
            doc.add_heading(line[2:], level=1)
        elif line.startswith("## "):  # Subsection Heading
            doc.add_heading(line[3:], level=2)
        else:  # Normal paragraph
            doc.add_paragraph(line)
            # paragraph = doc.add_paragraph(line)
            # parts = re.split(r"(\*\*(.*?)\*\*)", line)  # Split by **bold text**
            # print(parts)
            # for part in parts:
            #     if part.startswith("**") and part.endswith("**"):
            #         paragraph.add_run(part[2:-2]).bold = True  # Remove ** before adding bold text
            #     elif part:  # Normal text
            #         paragraph.add_run(part)

    # Save document
    file_path = "generated_documentation.docx"
    doc.save(file_path)
    return file_path  # Return file path for download

In [ ]:
file = write_to_docx(document_response)

NameError: name 'write_to_docx' is not defined

In [62]:
# define mainchain again as a graph
def retrieve(state):
  retrieved_docs = qdrant_retriever.invoke(state["query"])
  return {"context" : retrieved_docs}

def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = main_llm_prompt.format_messages(query=state["query"], context=docs_content, code_language=state['code_language'])
  response = main_llm.invoke(messages)
  return {"response" : response.content}

class State(TypedDict):
  query: str
  code_language: str
  context: List[Document]
  response: str

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
main_graph = graph_builder.compile()

# answer = graph.invoke({"query" : 'what is the use of function "generate_student_grades"', 'code_language': 'Python' })
# answer

In [ ]:
docs

[Document(metadata={'type': 'Imported Packages', 'name': 'Imported Packages', 'content': '{"Imported_Packages": [\n    {"name": "os", "desc": "A standard library that provides a way of using operating system-dependent functionality like reading or writing to the file system."},\n    {"name": "getpass", "desc": "A standard library that provides a way to securely handle password prompts without echoing input."},\n    {"name": "operator", "desc": "A standard library that exports a set of functions corresponding to the intrinsic operators of Python."},\n    {"name": "langchain_openai", "desc": "A library that provides tools for building applications with OpenAI\'s language processing capabilities."},\n    {"name": "langgraph", "desc": "I don\'t know details about this package"},\n    {"name": "typing_extensions", "desc": "A backport of recent additions to the typing module in Python, providing support for type hints."},\n    {"name": "langchain_core", "desc": "I don\'t know details about t

# Generate Golden Dataset using RAGAS

In [54]:
chunks

{'Imports': {'type': 'imports',
  'description': [{'statistics': 'A standard library in Python that provides functions for calculating mathematical statistics of numeric data. This package is used for computing metrics like mean, median, and other statistical measures on the student grades.'},
   {'random': 'A standard library in Python that implements pseudo-random number generators for various distributions. This package is used to generate random integers for student grades within a specified range.'}]},
 'Functions': [{'type': 'function',
   'name': 'generate_student_grades',
   'description': 'Generates a list of random student grades based on the number of students specified. It takes an integer as input representing the number of students and returns a list of integers representing grades, which are randomly generated between 50 and 100.'},
  {'type': 'function',
   'name': 'calculate_grade_stats',
   'description': 'Calculates and returns statistical metrics for a given list of

In [55]:
from langchain.schema import Document
# docs = [
#     Document(
#         page_content=f"{chunk.get('type', '')} - {chunk.get('name', '')} - {chunk.get('description', '')}",  # Content for the model
#         metadata={**chunk}  # Store metadata, but don't put embeddings here
#     )
#     for chunk in chunks_list
# ]
docs2 = [
    Document(
        page_content=f"{chunks[key]}",  # Content for the model
        metadata={'Type': key}  # Store metadata, but don't put embeddings here
    )
    for key in chunks
    ]

# for key in chunks:
#     if isinstance(chunks[key], dict):
#         chunks_list.append(chunks[key])
#     elif isinstance(chunks[key], list):
#         for value in chunks[key]:
#             chunks_list.append(value)
docs2

[Document(metadata={'Type': 'Imports'}, page_content="{'type': 'imports', 'description': [{'statistics': 'A standard library in Python that provides functions for calculating mathematical statistics of numeric data. This package is used for computing metrics like mean, median, and other statistical measures on the student grades.'}, {'random': 'A standard library in Python that implements pseudo-random number generators for various distributions. This package is used to generate random integers for student grades within a specified range.'}]}"),
 Document(metadata={'Type': 'Functions'}, page_content="[{'type': 'function', 'name': 'generate_student_grades', 'description': 'Generates a list of random student grades based on the number of students specified. It takes an integer as input representing the number of students and returns a list of integers representing grades, which are randomly generated between 50 and 100.'}, {'type': 'function', 'name': 'calculate_grade_stats', 'descriptio

In [35]:
docs2[0].metadata

{'file_name': 'code.py'}

In [57]:
ragas_docs = [
    Document(
        page_content=f"{doc.page_content}",  # Extract actual content
        metadata={"source": doc.metadata['Type']}  # Define a unique source
    )
    for doc in docs2
]

ragas_docs

[Document(metadata={'source': 'Imports'}, page_content="{'type': 'imports', 'description': [{'statistics': 'A standard library in Python that provides functions for calculating mathematical statistics of numeric data. This package is used for computing metrics like mean, median, and other statistical measures on the student grades.'}, {'random': 'A standard library in Python that implements pseudo-random number generators for various distributions. This package is used to generate random integers for student grades within a specified range.'}]}"),
 Document(metadata={'source': 'Functions'}, page_content="[{'type': 'function', 'name': 'generate_student_grades', 'description': 'Generates a list of random student grades based on the number of students specified. It takes an integer as input representing the number of students and returns a list of integers representing grades, which are randomly generated between 50 and 100.'}, {'type': 'function', 'name': 'calculate_grade_stats', 'descri

In [1]:
# from langchain.schema import Document
# ragas_docs_2 = [
#     Document(
#         page_content=f"{doc.metadata.get('type', '')} - {doc.metadata.get('name', '')} - {doc.metadata.get('description', '')}",  # Extract actual content
#         metadata={"source": f"{doc.metadata.get('type', '')} - {doc.metadata.get('name', '')}"}  # Define a unique source
#     )
#     for doc in docs
# ]
# ragas_docs_2


NameError: name 'docs' is not defined

In [58]:
# generate test questions
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset import TestsetGenerator


generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(ragas_docs, testset_size=15)
dataset.to_pandas()


Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/3 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/7 [00:00<?, ?it/s]

Applying OverlapScoreBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Wat is the use of random integers in data anal...,"[{'type': 'imports', 'description': [{'statist...",Random integers are used to generate random nu...,single_hop_specifc_query_synthesizer
1,Wht are the uses of Python in data analysis?,"[{'type': 'imports', 'description': [{'statist...",Python is used in data analysis for its standa...,single_hop_specifc_query_synthesizer
2,How are random integers used in analyzing stud...,"[{'type': 'imports', 'description': [{'statist...",Random integers are generated using a standard...,single_hop_specifc_query_synthesizer
3,Wht is the median in student grades?,"[{'type': 'imports', 'description': [{'statist...",The median is a statistical measure that can b...,single_hop_specifc_query_synthesizer
4,Can you tell me what Python is used for in ana...,"[{'type': 'imports', 'description': [{'statist...",Python is used in analyzing student performanc...,single_hop_specifc_query_synthesizer
5,How does the function calculate_grade_stats wo...,"[[{'type': 'function', 'name': 'generate_stude...",The function calculate_grade_stats calculates ...,single_hop_specifc_query_synthesizer
6,What does the main function do in the context ...,"[[{'type': 'function', 'name': 'generate_stude...",The main function orchestrates the generation ...,single_hop_specifc_query_synthesizer
7,What does calculate_grade_stats do?,"[[{'type': 'function', 'name': 'generate_stude...",calculate_grade_stats calculates and returns s...,single_hop_specifc_query_synthesizer
8,How do you use the function calculate_grade_st...,"[[{'type': 'function', 'name': 'generate_stude...",The function calculate_grade_stats is designed...,single_hop_specifc_query_synthesizer
9,What is the role of the main function in the c...,"[[{'type': 'function', 'name': 'generate_stude...",The main function orchestrates the generation ...,single_hop_specifc_query_synthesizer


In [59]:
golden_dataset = dataset.to_pandas()
golden_dataset

,user_input,reference_contexts,reference,synthesizer_name
0,Wat is the use of random integers in data anal...,"[{'type': 'imports', 'description': [{'statist...",Random integers are used to generate random nu...,single_hop_specifc_query_synthesizer
1,Wht are the uses of Python in data analysis?,"[{'type': 'imports', 'description': [{'statist...",Python is used in data analysis for its standa...,single_hop_specifc_query_synthesizer
2,How are random integers used in analyzing stud...,"[{'type': 'imports', 'description': [{'statist...",Random integers are generated using a standard...,single_hop_specifc_query_synthesizer
3,Wht is the median in student grades?,"[{'type': 'imports', 'description': [{'statist...",The median is a statistical measure that can b...,single_hop_specifc_query_synthesizer
4,Can you tell me what Python is used for in ana...,"[{'type': 'imports', 'description': [{'statist...",Python is used in analyzing student performanc...,single_hop_specifc_query_synthesizer
5,How does the function calculate_grade_stats wo...,"[[{'type': 'function', 'name': 'generate_stude...",The function calculate_grade_stats calculates ...,single_hop_specifc_query_synthesizer
6,What does the main function do in the context ...,"[[{'type': 'function', 'name': 'generate_stude...",The main function orchestrates the generation ...,single_hop_specifc_query_synthesizer
7,What does calculate_grade_stats do?,"[[{'type': 'function', 'name': 'generate_stude...",calculate_grade_stats calculates and returns s...,single_hop_specifc_query_synthesizer
8,How do you use the function calculate_grade_st...,"[[{'type': 'function', 'name': 'generate_stude...",The function calculate_grade_stats is designed...,single_hop_specifc_query_synthesizer
9,What is the role of the main function in the c...,"[[{'type': 'function', 'name': 'generate_stude...",The main function orchestrates the generation ...,single_hop_specifc_query_synthesizer


In [60]:
golden_dataset.to_csv('golden_dataset.csv')

# Evaluate solution using RAGAS

In [67]:
golden_dataset

,user_input,reference_contexts,reference,synthesizer_name
0,Wat is the use of random integers in data anal...,"[{'type': 'imports', 'description': [{'statist...",Random integers are used to generate random nu...,single_hop_specifc_query_synthesizer
1,Wht are the uses of Python in data analysis?,"[{'type': 'imports', 'description': [{'statist...",Python is used in data analysis for its standa...,single_hop_specifc_query_synthesizer
2,How are random integers used in analyzing stud...,"[{'type': 'imports', 'description': [{'statist...",Random integers are generated using a standard...,single_hop_specifc_query_synthesizer
3,Wht is the median in student grades?,"[{'type': 'imports', 'description': [{'statist...",The median is a statistical measure that can b...,single_hop_specifc_query_synthesizer
4,Can you tell me what Python is used for in ana...,"[{'type': 'imports', 'description': [{'statist...",Python is used in analyzing student performanc...,single_hop_specifc_query_synthesizer
5,How does the function calculate_grade_stats wo...,"[[{'type': 'function', 'name': 'generate_stude...",The function calculate_grade_stats calculates ...,single_hop_specifc_query_synthesizer
6,What does the main function do in the context ...,"[[{'type': 'function', 'name': 'generate_stude...",The main function orchestrates the generation ...,single_hop_specifc_query_synthesizer
7,What does calculate_grade_stats do?,"[[{'type': 'function', 'name': 'generate_stude...",calculate_grade_stats calculates and returns s...,single_hop_specifc_query_synthesizer
8,How do you use the function calculate_grade_st...,"[[{'type': 'function', 'name': 'generate_stude...",The function calculate_grade_stats is designed...,single_hop_specifc_query_synthesizer
9,What is the role of the main function in the c...,"[[{'type': 'function', 'name': 'generate_stude...",The main function orchestrates the generation ...,single_hop_specifc_query_synthesizer


In [78]:
openai_embedder_dataset = golden_dataset.copy()
openai_embedder_dataset['response'] = None
openai_embedder_dataset['retrieved_contexts'] = None

# for test_row in openai_embedder_dataset:
#   response = main_graph.invoke({"query" : test_row.eval_sample.user_input, 'code_language': 'Python'})
#   test_row.eval_sample.response = response["response"]
#   test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

for index, row in openai_embedder_dataset.iterrows():
    response = main_graph.invoke({"query": row.user_input, "code_language": "Python"})
    print(response)
    print(f'response \n {response["response"]}')
    print(f'retrieved_context \n {[context.page_content for context in response["context"]]}')
    openai_embedder_dataset.loc[index, 'response'] = response["response"]
    openai_embedder_dataset.at[index, 'retrieved_contexts'] = [context.page_content for context in response["context"]]

openai_embedder_dataset

{'query': 'Wat is the use of random integers in data analysis?', 'code_language': 'Python', 'context': [Document(metadata={'type': 'imports', 'description': [{'statistics': 'A standard library in Python that provides functions for calculating mathematical statistics of numeric data. This package is used for computing metrics like mean, median, and other statistical measures on the student grades.'}, {'random': 'A standard library in Python that implements pseudo-random number generators for various distributions. This package is used to generate random integers for student grades within a specified range.'}], '_id': 'ab72f2fb260445d0a7db49bf08f2cd05', '_collection_name': 'description_rag_data'}, page_content="imports -  - [{'statistics': 'A standard library in Python that provides functions for calculating mathematical statistics of numeric data. This package is used for computing metrics like mean, median, and other statistical measures on the student grades.'}, {'random': 'A standard

,user_input,reference_contexts,reference,synthesizer_name,response,retrieved_contexts
0,Wat is the use of random integers in data anal...,"[{'type': 'imports', 'description': [{'statist...",Random integers are used to generate random nu...,single_hop_specifc_query_synthesizer,I'm unable to answer that question as it is ou...,[imports - - [{'statistics': 'A standard libr...
1,Wht are the uses of Python in data analysis?,"[{'type': 'imports', 'description': [{'statist...",Python is used in data analysis for its standa...,single_hop_specifc_query_synthesizer,I'm unable to answer that question as it is ou...,[imports - - [{'statistics': 'A standard libr...
2,How are random integers used in analyzing stud...,"[{'type': 'imports', 'description': [{'statist...",Random integers are generated using a standard...,single_hop_specifc_query_synthesizer,The context provided does not cover the analys...,[function - generate_student_grades - Generate...
3,Wht is the median in student grades?,"[{'type': 'imports', 'description': [{'statist...",The median is a statistical measure that can b...,single_hop_specifc_query_synthesizer,The context does not provide specific values f...,[function - calculate_grade_stats - Calculates...
4,Can you tell me what Python is used for in ana...,"[{'type': 'imports', 'description': [{'statist...",Python is used in analyzing student performanc...,single_hop_specifc_query_synthesizer,"In analyzing student performance data, Python ...",[imports - - [{'statistics': 'A standard libr...
5,How does the function calculate_grade_stats wo...,"[[{'type': 'function', 'name': 'generate_stude...",The function calculate_grade_stats calculates ...,single_hop_specifc_query_synthesizer,The function `calculate_grade_stats` works by ...,[function - calculate_grade_stats - Calculates...
6,What does the main function do in the context ...,"[[{'type': 'function', 'name': 'generate_stude...",The main function orchestrates the generation ...,single_hop_specifc_query_synthesizer,The main function orchestrates the following t...,[function - main - The main execution function...
7,What does calculate_grade_stats do?,"[[{'type': 'function', 'name': 'generate_stude...",calculate_grade_stats calculates and returns s...,single_hop_specifc_query_synthesizer,The function `calculate_grade_stats` calculate...,[function - calculate_grade_stats - Calculates...
8,How do you use the function calculate_grade_st...,"[[{'type': 'function', 'name': 'generate_stude...",The function calculate_grade_stats is designed...,single_hop_specifc_query_synthesizer,To use the function `calculate_grade_stats` to...,[function - calculate_grade_stats - Calculates...
9,What is the role of the main function in the c...,"[[{'type': 'function', 'name': 'generate_stude...",The main function orchestrates the generation ...,single_hop_specifc_query_synthesizer,The main function in the context of student gr...,[function - main - The main execution function...


In [80]:
from ragas import EvaluationDataset

openai_evaluation_dataset = EvaluationDataset.from_pandas(openai_embedder_dataset)

# Choose the evaluator LLM
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
# Evaluate using RAGAS
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, LLMContextPrecisionWithReference
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=openai_evaluation_dataset,
    metrics=[Faithfulness(), FactualCorrectness(), LLMContextRecall(), LLMContextPrecisionWithReference(), ResponseRelevancy()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'faithfulness': 0.5686, 'factual_correctness': 0.6040, 'context_recall': 0.9000, 'llm_context_precision_with_reference': 0.9833, 'answer_relevancy': 0.5771}

In [ ]:
result.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,context_recall,faithfulness,factual_correctness,answer_relevancy,context_entity_recall,noise_sensitivity_relevant
0,Can you explain the role of langchain_core in ...,"[Imported Packages - Imported Packages - {""Imp...","[{""Imported_Packages"": [\n {""name"": ""os"", ""...",I don't know details about the langchain_core ...,The context does not provide specific details ...,1.0,1.000000,1.00,0.000000,0.00,0.0
1,What is the os package in Python and how does ...,"[Imported Packages - Imported Packages - {""Imp...","[{""Imported_Packages"": [\n {""name"": ""os"", ""...",The `os` package in Python is a standard libra...,The os package is a standard library in Python...,1.0,0.750000,0.86,0.924705,0.00,0.0
2,What typing_extensions do in Python?,"[Imported Packages - Imported Packages - {""Imp...","[{""Imported_Packages"": [\n {""name"": ""os"", ""...",The `typing_extensions` package in Python is a...,typing_extensions is a backport of recent addi...,1.0,0.500000,0.67,0.920737,0.25,0.0
3,Can you elaborate on the functionalities provi...,"[Imported Packages - Imported Packages - {""Imp...","[{""Imported_Packages"": [\n {""name"": ""os"", ""...",The 'os' package in Python provides a way to u...,The 'os' package is a standard library in Pyth...,0.5,0.157895,0.55,0.961260,0.00,0.0
4,What is the purpose of the langgraph package i...,"[Imported Packages - Imported Packages - {""Imp...","[{""Imported_Packages"": [\n {""name"": ""os"", ""...",The context does not provide details about the...,The context does not provide specific details ...,1.0,0.500000,0.67,0.000000,0.00,0.0
5,What is the purpose of the langgraph package i...,"[Imported Packages - Imported Packages - {""Imp...","[{""Imported_Packages"": [\n {""name"": ""os"", ""...",The context does not provide details about the...,The context does not provide specific details ...,1.0,1.000000,1.00,0.000000,0.00,0.0
6,What is the role of langchain_core in Python d...,"[Imported Packages - Imported Packages - {""Imp...","[{""Imported_Packages"": [\n {""name"": ""os"", ""...",The context does not provide details about the...,The context does not provide specific details ...,1.0,1.000000,1.00,0.000000,0.50,0.0
7,What langchain.schema.output_parser do?,"[Imported Packages - Imported Packages - {""Imp...","[{""Imported_Packages"": [\n {""name"": ""os"", ""...",I don't know details about the langchain.schem...,I don't know details about this package.,1.0,1.000000,0.00,0.000000,0.00,0.0
8,What the os package do in Python and how it he...,"[Imported Packages - Imported Packages - {""Imp...","[{""Imported_Packages"": [\n {""name"": ""os"", ""...",The `os` package in Python is a standard libra...,The os package is a standard library that prov...,1.0,0.363636,0.53,0.901486,0.00,0.0
9,Can you tell me what langchain_core is used fo...,"[Imported Packages - Imported Packages - {""Imp...","[{""Imported_Packages"": [\n {""name"": ""os"", ""...",I don't have details about the `langchain_core...,I don't know details about the langchain_core ...,1.0,0.500000,0.67,0.000000,0.00,0.5


# Use open source model and fine tune it

## Install Dependencies

In [18]:
!pip install -qU langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters

In [19]:
!pip install -qU faiss-cpu python-pptx==1.0.2 nltk==3.9.1 pymupdf beautifulsoup4 lxml

In [21]:
!pip install docx2txt


  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=dc7cb308fb71b446e89c4ca890914343ec2c8f19702466b1d45121ac4f86e6be
  Stored in directory: /root/.cache/pip/wheels/0f/0e/7a/3094a4ceefe657bff7e12dd9592a9d5b6487ef4338ace0afa6
Successfully built docx2txt


## Load data

In [30]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import BSHTMLLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_community.document_loaders import Docx2txtLoader




# path = "data/"
# text_loader = DirectoryLoader(path, glob="*.html", loader_cls=BSHTMLLoader)

# loader = UnstructuredWordDocumentLoader('/content/generated_documentation.docx')
# document = loader.load()

loader = Docx2txtLoader('/content/generated_documentation.docx')
document = loader.load()

document

[Document(metadata={'source': '/content/generated_documentation.docx'}, page_content="**Python Script for Generating and Analyzing Student Grades**\n\n\n\n**Imports**\n\n\n\nThe following libraries are utilized in this Python script:\n\n\n\n- statistics:   \n\n  - Purpose: This module provides functions to calculate mathematical statistics of numeric data.  \n\n  - Use in Script: It is used to compute statistical metrics such as the mean and median of the student grades generated.\n\n\n\n- random:  \n\n  - Purpose: This module implements pseudo-random number generators for various distributions.  \n\n  - Use in Script: It is employed to generate random grades for students within the specified range of 50 to 100.\n\n\n\n**Functions**\n\n\n\n**1. generate_student_grades(num_students)**\n\n\n\n- Purpose: This function generates a list of random integers representing student grades.  \n\n- Parameters:  \n\n  - `num_students` (int): Specifies the number of student grades to generate.  \n\n-

In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 17,
    length_function = len
)

In [32]:
training_documents = text_splitter.split_documents(document)
training_documents

[Document(metadata={'source': '/content/generated_documentation.docx'}, page_content='**Python Script for Generating and Analyzing Student Grades**\n\n\n\n**Imports**\n\n\n\nThe following libraries are utilized in this Python script:\n\n\n\n- statistics:   \n\n  - Purpose: This module provides functions to calculate mathematical statistics of numeric data.  \n\n  - Use in Script: It is used to compute statistical metrics such as the mean and median of the student grades generated.\n\n\n\n- random:  \n\n  - Purpose: This module implements pseudo-random number generators for various distributions.'),
 Document(metadata={'source': '/content/generated_documentation.docx'}, page_content='- Use in Script: It is employed to generate random grades for students within the specified range of 50 to 100.\n\n\n\n**Functions**\n\n\n\n**1. generate_student_grades(num_students)**\n\n\n\n- Purpose: This function generates a list of random integers representing student grades.  \n\n- Parameters:  \n\n  

In [33]:
len(training_documents)

10

In [34]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

In [40]:
training_split_documents = training_documents[:6]
val_split_documents = training_documents[6:8]
test_split_documents = training_documents[8:10]

## Construct finetuning dataset

In [44]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

question_generation_chain = qa_prompt_template | qa_chat_model



In [45]:
from tqdm import tqdm
import uuid
import asyncio


async def process_document(document, n_questions):
  questions_generated = await question_generation_chain.ainvoke({"context": document.page_content, "n_questions":n_questions})

  doc_questions = {}
  doc_relevant_docs = {}

  for question in questions_generated.content.split("\n"):
    question_id = str(uuid.uuid4())
    doc_questions[question_id] = "".join(question.split(".")[1:]).strip()
    doc_relevant_docs[question_id] = [document.metadata["id"]]

  return doc_questions, doc_relevant_docs

async def create_questions(documents, n_questions):
  tasks = [process_document(doc, n_questions) for doc in documents]

  questions = {}
  relevant_docs = {}

  for task in tqdm(asyncio.as_completed(tasks), total=len(documents), desc="Processing Document"):
    doc_questions, doc_relevant_docs = await task
    questions.update(doc_questions)
    relevant_docs.update(doc_relevant_docs)

  return questions, relevant_docs

In [47]:
training_questions, training_relevant_contexts = await create_questions(training_split_documents, 4)

Processing Document: 100%|██████████| 6/6 [00:01<00:00,  4.11it/s]


In [48]:
val_questions, val_relevant_contexts = await create_questions(val_split_documents, 4)

Processing Document: 100%|██████████| 2/2 [00:05<00:00,  2.79s/it]


In [49]:
test_questions, test_relevant_contexts = await create_questions(test_split_documents, 4)

Processing Document: 100%|██████████| 2/2 [00:01<00:00,  1.38it/s]


In [50]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)


test_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : test_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)



## Install 'snowflake' and set things us to fine tune it

In [51]:
!pip install -qU sentence_transformers datasets pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 29.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.1 which is incompatible.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.1 which is incompatible.


In [52]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-l"
model = SentenceTransformer(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/85.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [53]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

In [54]:
BATCH_SIZE = 10

corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)
examples

In [55]:
for query_id, query in queries.items():
    print(query)
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    print(text)
    print("****************************")

What is the purpose of the `generate_student_grades` function?
- Use in Script: It is employed to generate random grades for students within the specified range of 50 to 100.



**Functions**



**1. generate_student_grades(num_students)**



- Purpose: This function generates a list of random integers representing student grades.  

- Parameters:  

  - `num_students` (int): Specifies the number of student grades to generate.  

- Return Value:
****************************
What parameter does the `generate_student_grades` function require?
- Use in Script: It is employed to generate random grades for students within the specified range of 50 to 100.



**Functions**



**1. generate_student_grades(num_students)**



- Purpose: This function generates a list of random integers representing student grades.  

- Parameters:  

  - `num_students` (int): Specifies the number of student grades to generate.  

- Return Value:
****************************
What range of values does the functio

In [56]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)


In [57]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [58]:
EPOCHS = 10

## Finetune base model

In [59]:
import wandb
wandb.init(mode="disabled")

warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic_ft',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
3,No log,No log,0.750000,1.000000,1.000000,1.000000,0.750000,0.333333,0.200000,0.100000,0.750000,1.000000,1.000000,1.000000,0.907732,0.875000,0.875000
6,No log,No log,0.750000,1.000000,1.000000,1.000000,0.750000,0.333333,0.200000,0.100000,0.750000,1.000000,1.000000,1.000000,0.907732,0.875000,0.875000
9,No log,No log,0.750000,1.000000,1.000000,1.000000,0.750000,0.333333,0.200000,0.100000,0.750000,1.000000,1.000000,1.000000,0.907732,0.875000,0.875000
12,No log,No log,0.750000,1.000000,1.000000,1.000000,0.750000,0.333333,0.200000,0.100000,0.750000,1.000000,1.000000,1.000000,0.907732,0.875000,0.875000
15,No log,No log,0.750000,1.000000,1.000000,1.000000,0.750000,0.333333,0.200000,0.100000,0.750000,1.000000,1.000000,1.000000,0.907732,0.875000,0.875000
18,No log,No log,0.750000,1.000000,1.000000,1.000000,0.750000,0.333333,0.200000,0.100000,0.750000,1.000000,1.000000,1.000000,0.907732,0.875000,0.875000
21,No log,No log,0.750000,1.000000,1.000000,1.000000,0.750000,0.333333,0.200000,0.100000,0.750000,1.000000,1.000000,1.000000,0.907732,0.875000,0.875000
24,No log,No log,0.750000,1.000000,1.000000,1.000000,0.750000,0.333333,0.200000,0.100000,0.750000,1.000000,1.000000,1.000000,0.907732,0.875000,0.875000
27,No log,No log,0.750000,1.000000,1.000000,1.000000,0.750000,0.333333,0.200000,0.100000,0.750000,1.000000,1.000000,1.000000,0.907732,0.875000,0.875000
30,No log,No log,0.750000,1.000000,1.000000,1.000000,0.750000,0.333333,0.200000,0.100000,0.750000,1.000000,1.000000,1.000000,0.907732,0.875000,0.875000


## Push tuned model to huggning face

In [61]:
from huggingface_hub import notebook_login

notebook_login()

hf_username = "melghorab"

model.push_to_hub(f"{hf_username}/snowflake-arctic-embed-l0-fineTuned")

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

'https://huggingface.co/melghorab/snowflake-arctic-embed-l0-fineTuned/commit/6049080f6b9a707bae60551ab2aac6f10fed5a36'

# Evaluate Tuned Model Vs Base Model

In [65]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

# import tqdm

from tqdm import tqdm

In [63]:
def evaluate_embedder(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

## Evaluate Base Model

In [66]:
from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")
arctic_embed_m_results = evaluate_embedder(test_dataset, huggingface_embeddings)

100%|██████████| 8/8 [00:04<00:00,  1.93it/s]


In [67]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

1.0

## Evaluate Fine Tuned Model

In [68]:
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm


# finetune_embeddings = HuggingFaceEmbeddings(model_name="finetuned_arctic_ft")
finetune_embeddings = HuggingFaceEmbeddings(model_name="melghorab/snowflake-arctic-embed-l0-fineTuned")

finetune_results = evaluate_embedder(test_dataset, finetune_embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

100%|██████████| 8/8 [00:05<00:00,  1.35it/s]


In [69]:
finetune_results_df = pd.DataFrame(finetune_results)
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

1.0

# Evaluate RAG with Finetuned Model Using RAGAS

## Insert Documents in Vector Store and reconstruct Main Chain using fine tuned embedder

In [75]:
import pickle
with open("/content/description_chunks.pkl", "rb") as pickle_file:
    docs = pickle.load(pickle_file)
docs

[Document(metadata={'type': 'imports', 'description': [{'statistics': 'A standard library in Python that provides functions for calculating mathematical statistics of numeric data. This package is used for computing metrics like mean, median, and other statistical measures on the student grades.'}, {'random': 'A standard library in Python that implements pseudo-random number generators for various distributions. This package is used to generate random integers for student grades within a specified range.'}]}, page_content="imports -  - [{'statistics': 'A standard library in Python that provides functions for calculating mathematical statistics of numeric data. This package is used for computing metrics like mean, median, and other statistical measures on the student grades.'}, {'random': 'A standard library in Python that implements pseudo-random number generators for various distributions. This package is used to generate random integers for student grades within a specified range.'}]

In [78]:
qdrant_client = QdrantClient(":memory:")  # Use in-memory or provide host/port

# # Define embedding model
# embedding_model = OpenAIEmbeddings(model=finetune_embeddings)

embedding_dim = 1024
qdrant_client.create_collection(
    collection_name="description_rag_data",
    vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
)

# Store in Qdrant with explicit embeddings
vectorstore = Qdrant(qdrant_client, collection_name="description_rag_data", embeddings=finetune_embeddings)
vectorstore.add_documents(docs)


# Define the retriever
qdrant_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

<ipython-input-78-568d1ddf869a>:13: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(qdrant_client, collection_name="description_rag_data", embeddings=finetune_embeddings)


## Reconstruct Rag graph



In [79]:
main_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
system_template = """You are an expert {code_language} developer.
Your role is to answer user's questions about code and its description that will be given to you in context

Rules for the output:
1. Don't answer out of context questions
2. Provide a single, clear response using only the given context
3. If needed, structure long responses in lists and sections

<context>
{context}
</context>
"""

human_template = "{query}"
main_llm_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

main_chain = (
    {"context": itemgetter("query") | qdrant_retriever, "code_language": itemgetter("code_language"), "query": itemgetter("query"), }
    | main_llm_prompt
    | main_llm
    | StrOutputParser()
)

# define mainchain again as a graph
def retrieve(state):
  retrieved_docs = qdrant_retriever.invoke(state["query"])
  return {"context" : retrieved_docs}

def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = main_llm_prompt.format_messages(query=state["query"], context=docs_content, code_language=state['code_language'])
  response = main_llm.invoke(messages)
  return {"response" : response.content}

class State(TypedDict):
  query: str
  code_language: str
  context: List[Document]
  response: str

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
main_graph = graph_builder.compile()


## Evaluate

In [80]:
finetuned_embedder_dataset = golden_dataset.copy()
finetuned_embedder_dataset['response'] = None
finetuned_embedder_dataset['retrieved_contexts'] = None

# for test_row in finetuned_embedder_dataset:
#   response = main_graph.invoke({"query" : test_row.eval_sample.user_input, 'code_language': 'Python'})
#   test_row.eval_sample.response = response["response"]
#   test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

for index, row in finetuned_embedder_dataset.iterrows():
    response = main_graph.invoke({"query": row.user_input, "code_language": "Python"})
    print(response)
    print(f'response \n {response["response"]}')
    print(f'retrieved_context \n {[context.page_content for context in response["context"]]}')
    finetuned_embedder_dataset.loc[index, 'response'] = response["response"]
    finetuned_embedder_dataset.at[index, 'retrieved_contexts'] = [context.page_content for context in response["context"]]

finetuned_embedder_dataset

,user_input,reference_contexts,reference,synthesizer_name,response,retrieved_contexts
0,Wat is the use of random integers in data anal...,"[""{'type': 'imports', 'description': [{'statis...",Random integers are used to generate random nu...,single_hop_specifc_query_synthesizer,I'm unable to answer that question as it is ou...,[imports - - [{'statistics': 'A standard libr...
1,Wht are the uses of Python in data analysis?,"[""{'type': 'imports', 'description': [{'statis...",Python is used in data analysis for its standa...,single_hop_specifc_query_synthesizer,I'm unable to answer that question as it is ou...,[imports - - [{'statistics': 'A standard libr...
2,How are random integers used in analyzing stud...,"[""{'type': 'imports', 'description': [{'statis...",Random integers are generated using a standard...,single_hop_specifc_query_synthesizer,Random integers are used in analyzing student ...,[imports - - [{'statistics': 'A standard libr...
3,Wht is the median in student grades?,"[""{'type': 'imports', 'description': [{'statis...",The median is a statistical measure that can b...,single_hop_specifc_query_synthesizer,The median in student grades is a statistical ...,[function - calculate_grade_stats - Calculates...
4,Can you tell me what Python is used for in ana...,"[""{'type': 'imports', 'description': [{'statis...",Python is used in analyzing student performanc...,single_hop_specifc_query_synthesizer,"In analyzing student performance data, Python ...",[imports - - [{'statistics': 'A standard libr...
5,How does the function calculate_grade_stats wo...,"[""[{'type': 'function', 'name': 'generate_stud...",The function calculate_grade_stats calculates ...,single_hop_specifc_query_synthesizer,The function `calculate_grade_stats` works as ...,[function - calculate_grade_stats - Calculates...
6,What does the main function do in the context ...,"[""[{'type': 'function', 'name': 'generate_stud...",The main function orchestrates the generation ...,single_hop_specifc_query_synthesizer,The main function orchestrates the following t...,[function - main - The main execution function...
7,What does calculate_grade_stats do?,"[""[{'type': 'function', 'name': 'generate_stud...",calculate_grade_stats calculates and returns s...,single_hop_specifc_query_synthesizer,The function `calculate_grade_stats` calculate...,[function - calculate_grade_stats - Calculates...
8,How do you use the function calculate_grade_st...,"[""[{'type': 'function', 'name': 'generate_stud...",The function calculate_grade_stats is designed...,single_hop_specifc_query_synthesizer,To use the function `calculate_grade_stats` to...,[function - calculate_grade_stats - Calculates...
9,What is the role of the main function in the c...,"[""[{'type': 'function', 'name': 'generate_stud...",The main function orchestrates the generation ...,single_hop_specifc_query_synthesizer,The main function in the context of student gr...,[function - main - The main execution function...


In [83]:
import ast
finetuned_embedder_dataset["reference_contexts"] = finetuned_embedder_dataset["reference_contexts"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [84]:
from ragas import EvaluationDataset

finetuned_dataset = EvaluationDataset.from_pandas(finetuned_embedder_dataset)

# Choose the evaluator LLM
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
# Evaluate using RAGAS
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, LLMContextPrecisionWithReference
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=finetuned_dataset,
    metrics=[Faithfulness(), FactualCorrectness(), LLMContextRecall(), LLMContextPrecisionWithReference(), ResponseRelevancy()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

{'faithfulness': 0.7250, 'factual_correctness': 0.5780, 'context_recall': 1.0000, 'llm_context_precision_with_reference': 1.0000, 'answer_relevancy': 0.7716}

In [73]:
golden_dataset = pd.read_csv('/content/golden_dataset (1).csv').drop(columns=['Unnamed: 0'])
golden_dataset

,user_input,reference_contexts,reference,synthesizer_name
0,Wat is the use of random integers in data anal...,"[""{'type': 'imports', 'description': [{'statis...",Random integers are used to generate random nu...,single_hop_specifc_query_synthesizer
1,Wht are the uses of Python in data analysis?,"[""{'type': 'imports', 'description': [{'statis...",Python is used in data analysis for its standa...,single_hop_specifc_query_synthesizer
2,How are random integers used in analyzing stud...,"[""{'type': 'imports', 'description': [{'statis...",Random integers are generated using a standard...,single_hop_specifc_query_synthesizer
3,Wht is the median in student grades?,"[""{'type': 'imports', 'description': [{'statis...",The median is a statistical measure that can b...,single_hop_specifc_query_synthesizer
4,Can you tell me what Python is used for in ana...,"[""{'type': 'imports', 'description': [{'statis...",Python is used in analyzing student performanc...,single_hop_specifc_query_synthesizer
5,How does the function calculate_grade_stats wo...,"[""[{'type': 'function', 'name': 'generate_stud...",The function calculate_grade_stats calculates ...,single_hop_specifc_query_synthesizer
6,What does the main function do in the context ...,"[""[{'type': 'function', 'name': 'generate_stud...",The main function orchestrates the generation ...,single_hop_specifc_query_synthesizer
7,What does calculate_grade_stats do?,"[""[{'type': 'function', 'name': 'generate_stud...",calculate_grade_stats calculates and returns s...,single_hop_specifc_query_synthesizer
8,How do you use the function calculate_grade_st...,"[""[{'type': 'function', 'name': 'generate_stud...",The function calculate_grade_stats is designed...,single_hop_specifc_query_synthesizer
9,What is the role of the main function in the c...,"[""[{'type': 'function', 'name': 'generate_stud...",The main function orchestrates the generation ...,single_hop_specifc_query_synthesizer
